In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
test = pd.read_csv('../data/survey_data_a_test.csv')
train = pd.read_csv('../data/survey_data_a_train.csv')

data_a = pd.concat([t])

In [2]:
raw_data = pd.read_excel('../data/ISI,ESS,PSGdata_20220113.xlsx')
raw_data.head()

/var/folders/l6/j5nwgjg11zqbq0y11b2myyb00000gn/T/ipykernel_37718/2156600217.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  raw_data = pd.read_excel('../data/ISI,ESS,PSGdata_20220113.xlsx')


,ID,PSG_date,PSG_ID,sex,age,birth,Final Diagnosis,OSA,insomnia,"PLMD, RLS",...,PVC(y/n),Bradycardia(y/n),Others(y/n),PLM#,Total \nLMI =PLMI(/h),Total LM \nArousal#,Total LM AI(/h),PLM \nArousal#,PLM \nAI(/h),MAI(/h)
0,1,2014-12-10,N14005,1,52,1962-02-26,sleep talking,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2014-12-10,PE141015,0,65,1948-12-17,RLS/OSA,1,0,1,...,NaN,NaN,NaN,NaN,16.7,NaN,NaN,NaN,NaN,NaN
2,3,2014-12-10,PE141017,0,53,1961-02-28,OSA,1,0,0,...,NaN,NaN,NaN,NaN,10.7,NaN,NaN,NaN,NaN,NaN
3,4,2014-12-10,PE141014,0,53,1961-06-01,OSA/PPI,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2015-03-31,PE150253,1,55,1959-12-19,OSA,1,0,0,...,N,N,N,22.0,3.2,2.0,0.3,0.0,0.0,0.3


In [3]:
ess = raw_data.iloc[:, 15:23].copy()
isi = raw_data.iloc[:, 23:30].copy()
ssq = raw_data.iloc[:, 30:39].copy()
etc = raw_data.iloc[:, 39:60].copy()
targets = raw_data.iloc[:, 7:15].copy()

In [4]:
pi = raw_data.iloc[:, 3:6].copy()

In [5]:
survey_data = pd.concat([pi, isi, ess, ssq, etc, targets], axis=1)
survey_data.head()

,sex,age,birth,ISI1a,ISI1b,ISI1c,ISI2,ISI3,ISI4,ISI5,...,검사당일\n음주여부(Y/N),검사당일\n음주량,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control
0,1,52,1962-02-26,1,0,0,0.0,4.0,4,4,...,NaN,NaN,0,0,0,0,0,0,0,1
1,0,65,1948-12-17,4,4,4,4.0,1.0,2,4,...,NaN,NaN,1,0,1,0,0,0,0,0
2,0,53,1961-02-28,0,1,0,2.0,2.0,2,2,...,NaN,NaN,1,0,0,0,0,0,0,0
3,0,53,1961-06-01,3,2,2,3.0,3.0,3,3,...,NaN,NaN,1,1,0,0,0,0,0,0
4,1,55,1959-12-19,4,4,4,4.0,4.0,2,4,...,N,NaN,1,0,0,0,0,0,0,0


In [6]:
print(survey_data.shape)

survey_missings = survey_data.isnull().sum()
survey_missings

(5481, 56)


sex                                        0
age                                        0
birth                                      0
ISI1a                                      0
ISI1b                                      0
ISI1c                                      0
ISI2                                       0
ISI3                                       0
ISI4                                       0
ISI5                                       0
ESS1                                       0
ESS2                                       0
ESS3                                       0
ESS4                                       0
ESS5                                       0
ESS6                                       0
ESS7                                       0
ESS8                                       0
SSQ1                                       0
SSQ2                                       0
SSQ3                                       0
SSQ4                                       0
SSQ5      

In [7]:
survey_data = survey_data.drop(['수면제이름', '검사당일\n음주량'], axis=1)
survey_data.shape

(5481, 54)

In [9]:
na_cols_list = []
for col in survey_data.columns:
    na_row = survey_data[survey_data[col] == 999]
    if len(na_row) != 0:
        na_cols_list.append(col)
        print(col, '\t', len(na_row))

ESS1 	 17
ESS2 	 17
ESS3 	 17
ESS4 	 17
ESS5 	 17
ESS6 	 17
ESS7 	 17
ESS8 	 17
SSQ1 	 3083
SSQ2 	 3116
SSQ3 	 3074
SSQ4 	 3080
SSQ5 	 3084
SSQ6 	 3127
SSQ7 	 3125
SSQ8 	 3087
SSQ9 	 3076
신장 	 5
체중 	 8
BMI 	 12
PSQI 	 1243
ISI 	 1143
SSS 	 1116
ESS 	 1104
HADS-A 	 5199
HADS-D 	 5200
K-BDI2 	 1247
WHO-QOL 	 3703
배둘레 	 1193
엉덩이둘레 	 1195
목둘레
앉은상태 	 1190
목둘레
누운상태 	 1192
머리둘레 	 1193
전날수면시간 	 891
OSA 	 1
insomnia 	 1
Catathrenia 	 1
control 	 1


In [9]:
survey_data = survey_data.drop(['HADS-A', 'HADS-D', 'WHO-QOL'], axis=1)
survey_data.shape

(5481, 51)

In [31]:
survey_copy = survey_data.copy()

In [32]:
survey_copy = survey_copy.drop(['PSQI', 'K-BDI2', '배둘레', '엉덩이둘레', '목둘레\n앉은상태', '목둘레\n누운상태', '머리둘레'], axis=1)

In [12]:
na_cols_list.remove('HADS-A')
na_cols_list.remove('HADS-D')
na_cols_list.remove('WHO-QOL')
na_cols_list.remove('PSQI')
na_cols_list.remove('K-BDI2')

drop_rows = []
for col in na_cols_list:
    drop_rows += list(survey_copy[survey_copy[col] == 999].index)

drop_rows = list(set(drop_rows))
print(len(drop_rows))

3440


In [13]:
survey_copy = survey_copy.drop(drop_rows, axis=0)
survey_copy.shape

(2041, 49)

In [20]:
survey_data = survey_data.drop(drop_rows, axis=0)
survey_data.shape

(1937, 51)

In [14]:
survey_copy.to_csv('../data/survey_data_b.csv', index=False)

In [15]:
from sklearn.model_selection import train_test_split

In [18]:
survey_copy.iloc[:, 41:51]

,OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control
77,1,0,0,0,0,0,0,0
79,1,0,0,0,0,0,0,0
80,1,0,0,0,0,0,0,0
81,0,0,0,0,0,0,0,1
83,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
5474,1,0,0,0,0,0,0,0
5475,0,0,1,0,0,0,0,0
5476,1,0,0,0,0,0,0,0
5477,1,0,0,0,0,0,0,0


In [20]:
target = survey_copy.iloc[:, 41:51]
features = survey_copy.drop(survey_copy.iloc[:, 41:51], axis=1)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

In [21]:
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [22]:
yorn_cols = [col for col in train.columns if 'Y/N' or 'y/n' in col]

for c in yorn_cols:
    train.loc[train[c] == 'Y', c] = 1
    train.loc[train[c] == 'y', c] = 1
    train.loc[train[c] == 'N', c] = 0
    train.loc[train[c] == 'n', c] = 0

In [23]:
yorn_cols = [col for col in test.columns if 'Y/N' or 'y/n' in col]

for c in yorn_cols:
    test.loc[test[c] == 'Y', c] = 1
    test.loc[test[c] == 'y', c] = 1
    test.loc[test[c] == 'N', c] = 0
    test.loc[test[c] == 'n', c] = 0

In [24]:
train = train.drop(['birth'], axis=1)
test = test.drop(['birth'], axis=1)

In [25]:
train[(train['SSQ2'] == ' ') | (train['SSQ2'] == '-')]

,sex,age,ISI1a,ISI1b,ISI1c,ISI2,ISI3,ISI4,ISI5,ESS1,...,검사당일수면제\n복용여부(Y/N),검사당일\n음주여부(Y/N),OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control
1377,1,69,0,0,0,0.0,0.0,0,2,1,...,0,0,1,0,0,1,0,0,0,0


In [26]:
test[(test['SSQ2'] == ' ') | (test['SSQ2'] == '-')]

,sex,age,ISI1a,ISI1b,ISI1c,ISI2,ISI3,ISI4,ISI5,ESS1,...,검사당일수면제\n복용여부(Y/N),검사당일\n음주여부(Y/N),OSA,insomnia,"PLMD, RLS",parasomnia,circadian rhythm sleep-wake disorder,"Narcolepsy, EDS, Hypersomnia",Catathrenia,control


In [27]:
train = train.drop([1377], axis=0)
train['SSQ2'] = train['SSQ2'].astype(int)
test['SSQ2'] = test['SSQ2'].astype(int)

In [28]:
train[['ISI2', 'ISI3']] = train[['ISI2', 'ISI3']].astype(int)
test[['ISI2', 'ISI3']] = test[['ISI2', 'ISI3']].astype(int)

In [29]:
train[['검사당일수면제\n복용여부(Y/N)', '검사당일\n음주여부(Y/N)']] = train[['검사당일수면제\n복용여부(Y/N)', '검사당일\n음주여부(Y/N)']].astype(int)
test[['검사당일수면제\n복용여부(Y/N)', '검사당일\n음주여부(Y/N)']] = test[['검사당일수면제\n복용여부(Y/N)', '검사당일\n음주여부(Y/N)']].astype(int)

In [30]:
train.to_csv('../data/survey_data_b_train.csv', index=False)
test.to_csv('../data/survey_data_b_test.csv', index=False)